In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [6]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [7]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

123 documents
34 classes ['A+ve', 'AB+ve', 'B-ve', 'Diabetes ', 'Diet chart', 'Dr.Maha', 'Dr.Rajalakshmi ', 'Fever', 'Hyperthyroidism', 'Instant remedy', 'Location-chennai', 'Location-coimbatore', 'Location-vellore', 'O-ve', 'Specialist', 'acne', 'adverse_drug', 'age', 'allergy', 'appointment', 'appointment-location', 'blood donor', 'blood_pressure', 'blood_pressure_search', 'cold', 'goodbye', 'greeting', 'height', 'hospital_search', 'options', 'pharmacy_search', 'thanks', 'time', 'weight']
180 unique lemmatized words ["'s", ',', '103', '160', '168', '169', '175.3', '18', '25', '34', '45', '50', '5pm', '65', '6pm', '78', '7pm', '89', '90', '95', 'a', 'a+ve', 'ab', 'ab+ve', 'ache', 'acne', 'adverse', 'all', 'allergy', 'am', 'an', 'anyone', 'appointment', 'are', 'ariyalur', 'awesome', 'b', 'b-ve', 'be', 'behavior', 'big', 'black', 'blood', 'book', 'by', 'bye', 'can', 'causing', 'chart', 'chatting', 'check', 'chennai', 'coimbatore', 'cold', 'consult', 'controlling', 'could', 'data', 'day'

In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [9]:
import random

In [10]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


<ipython-input-10-14f4db91aa1f>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [12]:
from tensorflow.keras.optimizers import SGD

In [13]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
25/25 [==============================] - 4s 7ms/step - loss: 3.5616 - accuracy: 0.0000e+00
Epoch 2/200
25/25 [==============================] - 0s 4ms/step - loss: 3.5167 - accuracy: 0.0407
Epoch 3/200
25/25 [==============================] - 0s 4ms/step - loss: 3.4293 - accuracy: 0.0813
Epoch 4/200
25/25 [==============================] - 0s 4ms/step - loss: 3.3539 - accuracy: 0.1301
Epoch 5/200
25/25 [==============================] - 0s 3ms/step - loss: 3.2644 - accuracy: 0.1463
Epoch 6/200
25/25 [==============================] - 0s 4ms/step - loss: 3.2140 - accuracy: 0.1707
Epoch 7/200
25/25 [==============================] - 0s 4ms/step - loss: 3.0544 - accuracy: 0.1870
Epoch 8/200
25/25 [==============================] - 0s 4ms/step - loss: 2.9032 - accuracy: 0.2520
Epoch 9/200
25/25 [==============================] - 0s 6ms/step - loss: 2.8403 - accuracy: 0.2520
Epoch 10/200
25/25 [==============================] - 0s 4ms/step - loss: 2.6400 - accuracy: 0.3089
Epoch

Epoch 83/200
25/25 [==============================] - 0s 4ms/step - loss: 0.2510 - accuracy: 0.9187
Epoch 84/200
25/25 [==============================] - 0s 4ms/step - loss: 0.4101 - accuracy: 0.8537
Epoch 85/200
25/25 [==============================] - 0s 4ms/step - loss: 0.2969 - accuracy: 0.9106
Epoch 86/200
25/25 [==============================] - 0s 3ms/step - loss: 0.3372 - accuracy: 0.8943
Epoch 87/200
25/25 [==============================] - 0s 4ms/step - loss: 0.2836 - accuracy: 0.9268
Epoch 88/200
25/25 [==============================] - 0s 4ms/step - loss: 0.2744 - accuracy: 0.9106
Epoch 89/200
25/25 [==============================] - 0s 4ms/step - loss: 0.2124 - accuracy: 0.9350
Epoch 90/200
25/25 [==============================] - 0s 4ms/step - loss: 0.1879 - accuracy: 0.9512
Epoch 91/200
25/25 [==============================] - 0s 3ms/step - loss: 0.2801 - accuracy: 0.9106
Epoch 92/200
25/25 [==============================] - 0s 4ms/step - loss: 0.2091 - accuracy: 0.9106


25/25 [==============================] - 0s 4ms/step - loss: 0.1267 - accuracy: 0.9593
Epoch 164/200
25/25 [==============================] - 0s 4ms/step - loss: 0.1262 - accuracy: 0.9675
Epoch 165/200
25/25 [==============================] - 0s 5ms/step - loss: 0.1784 - accuracy: 0.9106
Epoch 166/200
25/25 [==============================] - 0s 4ms/step - loss: 0.1359 - accuracy: 0.9512
Epoch 167/200
25/25 [==============================] - 0s 4ms/step - loss: 0.0843 - accuracy: 0.9837
Epoch 168/200
25/25 [==============================] - 0s 4ms/step - loss: 0.1931 - accuracy: 0.9106
Epoch 169/200
25/25 [==============================] - 0s 3ms/step - loss: 0.1121 - accuracy: 0.9756
Epoch 170/200
25/25 [==============================] - 0s 4ms/step - loss: 0.1174 - accuracy: 0.9756
Epoch 171/200
25/25 [==============================] - 0s 4ms/step - loss: 0.1211 - accuracy: 0.9593
Epoch 172/200
25/25 [==============================] - 0s 4ms/step - loss: 0.1372 - accuracy: 0.9512
Epoc